In [1]:
# importing the libraries and 

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re 
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available else "cpu")

In [2]:
# loading the data and putting the SOS and EOS token  

SOS_token = 0
EOS_token = 1 

# building the word2index and index2word dictionary 

class Lang:
    def __init__(self, name):
        self.name = name 
        self.word2index = {}
        self.word2Count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2 
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2Count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2Count[word] += 1 

In [3]:
# We have to convert the Unicode characters to ASCII, make everything lowercase, and trim the punctuations 

def unicodeToAscii(s):
    return ''.join(
        c for c  in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase. trim and remove non-letter characters 

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s 

In [4]:
# read the data file and split the file into lines and then slplit the lines into pairs.

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")
    
    # Read the file and split into lines 
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize 
    pairs = [[normalizeString(s) for s in l.split('t')] for l in lines]
    
    # Reverse pairs, make Lang instances 
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
        
    return input_lang, output_lang, pairs
    

In [5]:
# since there are a lot of example sentences and we want to train something quickly,
# we'll trim the data set to only relatively short and simple sentences 
MAX_LENGTH = 10

eng_prefixes = (
    "i am", "i m",
    "he is", "he s",
    "she is", "she s",
    "you are", "you re",
    "we are", "we re",
    "they are", "they re",
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
'''
the full process for preparng the data is:

'''